In [1]:
# libraries
import pandas as pd

# classification dictionary
import os
import sys
sys.path.append(os.path.abspath("modules"))
import modules.dictionary as dict
from modules.classify import classify_keywords

# set your path here
os.chdir("input_data")
print(f'current directory: {os.getcwd()}')

current directory: /Users/tomokitakata/Desktop/research/dev/muldistribution/input_data


ベースの分類は東大病院の診療科構成を参考。  
- 腎臓・人工透析内科：これはこれで一つあるっぽい
- ペインクリニック：内科、麻酔科、整形外科に分類される場合がある。
- 平成 20 年 4 月 1 日以降、広告することが認められない診療科名「神経科」、「呼吸器科」、「消化器科」、「胃腸科」、「循環器科」、「皮膚泌尿器科」、「性病科」、「こう門科」、「気管食道科」

In [2]:
# target word(s) : class(es) assigned

override_map_2020 = {

    # ----内科・外科----
    ("訪問診療", "訪問診療科"): ["一般内科"],
    ("漢方", "漢方専門", "漢方科", "漢方（思）", "漢方（不）", "漢方医学科", "漢方診療科"): ["一般内科"], 
    ("総合診療", "総合診療科"): ['一般内科', '一般外科'],
    ("基本診療", "基本診療科"): ['一般内科', '一般外科'],

     # ----消化器内科・外科----
    ("消科", "消化器", "消火器科", "消化器科", "消化器科（胃）"): ["消化器内科", "消化器外科"],
    ("胃腸科", "胃腸下科", "胃腸器科", "消・胃腸科", "消胃", "胃腸科（消化器科）"): ["消化器内科", "消化器外科"],
    "食道科": ["消化器内科", "消化器外科"],
    ("消化器・胃腸科", "消化器胃腸科", "消化器科（胃腸科）"): ["消化器内科", '消化器外科'],

     # ----呼吸器内科・外科----
    "呼吸器科": ["呼吸器内科", '呼吸器外科'],
    ("気管科", "気管支科"): ['呼吸器内科'],
    ("気管食科", "気管食道科", "気道食道科"): ['呼吸器内科', '呼吸器外科', '消化器内科', '消化器外科'],

     # ----循環器内科・外科----
    ("循環器科", "循環器", "循環科"): ["循環器内科", "循環器外科"],
    "循呼": ["呼吸器内科", '呼吸器外科', "循環器内科", "循環器外科"],

    # ----精神・心療----
    "精心内": ["精神神経科", "心療内科"],#
    "子どものこころ診療科": ['心療内科'],
    "心身医療科": ['心療内科'],

    "女性不妊症": ['産婦人科'],
    "疼": ['麻酔科', '整形外科', 'リハビリテーション科'],
    "成分献血": ['病理診断科'],#
    
    # ----肝臓・胆嚢・膵臓----
    ("肝臓科", "腎臓病"): ['肝・胆・膵内科', '肝・胆・膵外科'],
    "胆のう": ['肝・胆・膵内科', '肝・胆・膵外科'],
    "肝臓・胆のう・膵臓科": ['肝・胆・膵内科', '肝・胆・膵外科'],

    # ----癌----
    "が内": ['臨床腫瘍科'], # おそらく「がん内科」

    # ----歯科----
    "矯科": ["歯科・口腔顎顔面外科"], #おそらく「矯正歯科」
    "噛み合わせ不良による頭痛": ['歯科・口腔顎顔面外科'],

    # ----腎臓・内分泌----
    ("腎臓", "腎臓病", "腎臓科"): ['腎臓・内分泌内科', '腎臓・内分泌外科'],
    "甲状腺診療科": ['腎臓・内分泌内科', '腎臓・内分泌外科'],
    "血液浄化": ['腎臓・内分泌内科'],
    ("透析", "透析科", "人工透析", "人工透析科", "人口透析科", "通院人工透析診療所", "循環器科（透析）") : ['腎臓・内分泌内科'],
    "高内": ["腎臓・内分泌内科"], # おそらく「高血圧・内分泌内科」
    "小腎": ['小児科', "腎臓・内分泌内科"], # おそらく 小児腎臓内科 or 小児腎臓外科
    "内高精": ["腎臓・内分泌内科", "泌尿器科"], # おそらく 内分泌・高血圧・生殖内科 or 生殖医療科 (精 = 精巣機能：男性不妊？)   
}

In [3]:
# 分類対象となるキーワードの例
uniq_dpt = pd.read_csv('2020.csv').iloc[:,1]
uniq_dpt = uniq_dpt.to_list()
uniq_dpt

['内',
 'リハ',
 '精',
 '心内',
 '小',
 '外',
 '整',
 '産婦',
 '耳',
 '皮',
 '脳神経内科',
 '循',
 '眼',
 '泌',
 '放',
 '麻',
 '神',
 '呼吸器内科',
 '消化器内科',
 '消化器外科',
 '循環器内科',
 '脳',
 '心外',
 '神内',
 '乳腺外科',
 '形',
 'リウ',
 '呼外',
 'ペインクリニック外科',
 '歯',
 '矯歯',
 '歯外',
 '病理診断科',
 '呼吸器外科',
 '心臓血管内科',
 'アレ',
 '病理',
 '婦',
 '緩和ケア内科',
 '循内',
 '消外',
 '肛外',
 '消内',
 '整外',
 '脳外',
 '耳咽',
 '呼内',
 '人透',
 '消',
 '胃',
 '児童',
 '思春期精神科',
 '形外',
 '救急科',
 '血液内科',
 '呼',
 '腎臓内科',
 '救急',
 '糖尿病内科',
 '腫瘍内科',
 '透内',
 '乳外',
 '矯',
 '小歯',
 '内視鏡内',
 '肝臓内科',
 '放射線診断科',
 '放射線治療科',
 '頭頸部外科',
 '腫瘍外科',
 '病理診断',
 '胃腸内科',
 '外科',
 '産',
 '血管外科',
 '神経精神',
 '小外',
 '循環器外科',
 '人工透析内科',
 '糖尿病内分泌科',
 '脳神経外科',
 '肛',
 '神経精神科',
 '頭頸部',
 '耳鼻咽喉科',
 'ペインクリニック',
 '病',
 '肛門外科',
 '血液',
 '糖尿病代謝内科',
 '内（消化器',
 '循環器',
 '呼吸器',
 '内分泌',
 '代謝',
 '糖尿病',
 '大腸内視鏡',
 '老人',
 '人工透析',
 '漢方）',
 '外（消化器',
 '末梢血管',
 '肛門',
 '老人）',
 '性',
 'ペインクリニック内科',
 '肛門内科',
 '内分泌内科',
 '感染症内科',
 '大腸外科',
 '美容皮膚科',
 '緩和ケア外科',
 '胃腸外科',
 '内視鏡内科',
 '内視鏡外科',
 '糖尿病内分泌内科',
 '代謝内科',
 '臨床検査科',
 '泌尿器科',
 '放射線診療科',
 '腫

In [4]:
classified_keywords = classify_keywords(
    keywords=uniq_dpt,
    classification_dict=dict.classification_dict,
    multiple=True,            # 複数マッチを許可
    override_map=override_map_2020,
    override_match_type="partial"   # override判定は部分一致
)

# 分類結果をDataFrameに整形して表示
max_length = max(len(values) for values in classified_keywords.values())
data = {
    cat: values + [""] * (max_length - len(values))
    for cat, values in classified_keywords.items()
}
result_df = pd.DataFrame(data)
display(result_df)

,内科,一般内科,呼吸器内科,消化器内科,循環器内科,腎臓・内分泌内科,糖尿病・代謝内科,血液・腫瘍内科,アレルギー科,リウマチ科,...,小児外科,産婦人科,精神神経科,放射線科,救急・集中治療科,臨床腫瘍科,病理診断科,緩和ケア科,鍼灸院,分類不可
0,脳神経内科,漢方）,呼吸器内科,消化器内科,循環器内科,腎臓内科,糖尿病内科,心臓血管内科,アレ,リウ,...,小児心臓血管外科,産婦,思春期精神科,放射線診断科,救急科,腫瘍内科,病理診断科,緩和ケア内科,,内
1,呼吸器内科,漢方内科,呼内,消化器内科,循環器内科,人工透析内科,糖尿病内分泌科,血液内科,アレルギー疾患）,リウマチ科,...,小児脳神経外科,婦,神経精神,放射線治療科,救急,腫瘍外科,病理,緩和ケア外科,,精
2,消化器内科,心内(漢方）,呼吸気内科,消化器外科,循内,腎臓内科（人工透析）,糖尿病代謝内科,腫瘍内科,ｱﾚ,膠原病内科,...,小児形成外科,産,神経精神科,放射線診療科,ペインクリニック内科・救急科,臨床検査科,病理診断,緩和ケ,,心内
3,循環器内科,人間ドック,内（呼吸器,消内,循環器外科,腎臓内科（透析）,代謝,血液腫瘍内科,アレルギー疾患内科,リウマチ内科,...,小児外科,婦（生殖医療）,老年精神科,放射線,小児救急科,血液腫瘍内科,病理診,緩和,,小
4,心臓血管内科,総合内科,内科（呼吸器,胃腸内科,循環器,甲状腺内科,糖尿病,小児血液腫瘍内科,小児アレルギー科,リウマチ,...,小児歯科・歯科口腔外科,婦人科（不妊治療,児童思春期精神科,放射線診断,救急医療,腫瘍精神科,理学診療科,緩和ケア科,,整
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
485,,,,,,,,,,,...,,,,,,,,,,
486,,,,,,,,,,,...,,,,,,,,,,
487,,,,,,,,,,,...,,,,,,,,,,
488,,,,,,,,,,,...,,,,,,,,,,


In [16]:
# result_df.to_csv('../2010_classified.csv', encoding='utf-8', index=True)

診療科の分類は、医療レセプトにおける厚生労働省が指定する区分を使用。
https://www.mhlw.go.jp/topics/2009/05/dl/tp0521-1a_0053.pdf